In [10]:
import numpy as np
import torch
import torch.nn as nn
import os
from PIL import Image
import matplotlib.pyplot as plt
import random
import pickle
import random

In [11]:
#crop map

def crop_map(filename):
    size = 90
    error = int(random.normalvariate(0,20))
    
    image = Image.open(filename)
    img = np.asarray(image)
    left = img.shape[0]/2-size + error
    right = img.shape[0]/2+size + error
    image_crop = image.crop((left,left,right,right))
    img = np.array(image_crop)

    for i in range(2*size):
        for j in range(2*size):
            if(img[i,j,0]==52):
                img[i,j,:]=[255,255,255]
            else:
                img[i,j,:]=[0,0,0]

    image_crop = Image.fromarray(img)
    image_final = image_crop.resize((50,50),Image.ANTIALIAS)
    img = np.array(image_final)
    img_sum = np.mean(a=img,axis=2,dtype=np.uint8)
    
#     plt.imshow(img_sum,cmap='gray')

    return img_sum

In [12]:
#crop image
def image_origin(filename):
    image = Image.open(filename)
    img = np.array(image)[:,:,0:3]
    
    return img

def crop_image_center(filename):
    image = Image.open(filename)
    up = 320
    down = 560
    left = 240
    right = 840
    crop1 = image.crop((left,up,right,down)).resize((200,80))

    img = np.array(crop1)[:,:,0:3]
    
#     plt.imshow(img)
#     plt.show()

    return img

def crop_image_left(filename):
    image = Image.open(filename)
    image_rot = image.rotate(0)
    up = 360
    down = 600
    left = 400
    right = 1000
    crop1 = image_rot.crop((left,up,right,down)).resize((200,80))

    img = np.array(crop1)[:,:,0:3]
    
#     plt.imshow(img)
#     plt.show()

    return img

def crop_image_right(filename):
    image = Image.open(filename)
    image_rot = image.rotate(0)
    up = 360
    down = 600
    left = 80
    right = 680
    crop1 = image_rot.crop((left,up,right,down)).resize((200,80))

    img = np.array(crop1)[:,:,0:3]
    
#     plt.imshow(img)
#     plt.show()

    return img

# crop_image_center(path+file_list[300]+'/'+imgs[2])
# crop_image_left(path+file_list[300]+'/'+imgs[3])
# crop_image_right(path+file_list[300]+'/'+imgs[1])

In [15]:
#dataset build

def dataset_build(data_path, image_folders):
    
    print("data build start")

    dataset_origin = []
    dataset_center = []
    dataset_L = []
    dataset_R = []
    dataset_map = []
    labelset = []

    for i,folder in enumerate(image_folders):
        filename = os.listdir(data_path + folder)
        
        label = 0
        for img in filename:
            types = img.split('_')
            label += float(types[1].split('.png')[0])
            
            if types[0] == 'center':
                img_o = image_origin(data_path + folder +'/'+img)
                img_c = crop_image_center(data_path + folder +'/'+ img)
            if types[0] == 'left':
                img_l = crop_image_left(data_path + folder +'/'+ img)
            if types[0] == 'right':
                img_r = crop_image_right(data_path + folder +'/'+ img)
            if types[0] == 'map':
                img_map = crop_map(data_path + folder +'/'+ img)
            
        dataset_origin.append(np.transpose(img_o,(2,1,0)))
        dataset_center.append(np.transpose(img_c,(2,1,0)))
        dataset_L.append(np.transpose(img_l,(2,1,0)))
        dataset_R.append(np.transpose(img_r,(2,1,0)))
        dataset_map.append(np.expand_dims(img_map,axis=0))
        labelset.append(label/len(filename))
        
        print('{}/{} finished'.format(i,len(image_folders)))

        
    return [torch.from_numpy(np.array(dataset_center)),
            torch.from_numpy(np.array(dataset_L)), 
            torch.from_numpy(np.array(dataset_R)),
            torch.from_numpy(np.array(dataset_map)),dataset_origin], torch.from_numpy(np.array(labelset))

        

In [16]:
# make dataset

data_path = 'data/'
image_folders = os.listdir(data_path)
random.shuffle(image_folders)

test_num = 50
dev_num = 550

test_dataset,test_labels = dataset_build(data_path, image_folders[0:test_num])
dev_dataset,dev_labels = dataset_build(data_path, image_folders[test_num:dev_num])
train_dataset,train_labels = dataset_build(data_path, image_folders[dev_num:])

pickle_file = 'end2end_dataset_v2.pickle'

try:
    f = open(pickle_file, 'wb')
    save = {
        'train_dataset': train_dataset,
        'train_labels': train_labels,
        'dev_dataset': dev_dataset,
        'dev_labels': dev_labels,
        'test_dataset': test_dataset,
        'test_labels': test_labels,
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise

data build start
0/50 finished
1/50 finished
2/50 finished
3/50 finished
4/50 finished
5/50 finished
6/50 finished
7/50 finished
8/50 finished
9/50 finished
10/50 finished
11/50 finished
12/50 finished
13/50 finished
14/50 finished
15/50 finished
16/50 finished
17/50 finished
18/50 finished
19/50 finished
20/50 finished
21/50 finished
22/50 finished
23/50 finished
24/50 finished
25/50 finished
26/50 finished
27/50 finished
28/50 finished
29/50 finished
30/50 finished
31/50 finished
32/50 finished
33/50 finished
34/50 finished
35/50 finished
36/50 finished
37/50 finished
38/50 finished
39/50 finished
40/50 finished
41/50 finished
42/50 finished
43/50 finished
44/50 finished
45/50 finished
46/50 finished
47/50 finished
48/50 finished
49/50 finished
data build start
0/500 finished
1/500 finished
2/500 finished
3/500 finished
4/500 finished
5/500 finished
6/500 finished
7/500 finished
8/500 finished
9/500 finished
10/500 finished
11/500 finished
12/500 finished
13/500 finished
14/500 finis

443/500 finished
444/500 finished
445/500 finished
446/500 finished
447/500 finished
448/500 finished
449/500 finished
450/500 finished
451/500 finished
452/500 finished
453/500 finished
454/500 finished
455/500 finished
456/500 finished
457/500 finished
458/500 finished
459/500 finished
460/500 finished
461/500 finished
462/500 finished
463/500 finished
464/500 finished
465/500 finished
466/500 finished
467/500 finished
468/500 finished
469/500 finished
470/500 finished
471/500 finished
472/500 finished
473/500 finished
474/500 finished
475/500 finished
476/500 finished
477/500 finished
478/500 finished
479/500 finished
480/500 finished
481/500 finished
482/500 finished
483/500 finished
484/500 finished
485/500 finished
486/500 finished
487/500 finished
488/500 finished
489/500 finished
490/500 finished
491/500 finished
492/500 finished
493/500 finished
494/500 finished
495/500 finished
496/500 finished
497/500 finished
498/500 finished
499/500 finished
data build start
0/6431 finishe

408/6431 finished
409/6431 finished
410/6431 finished
411/6431 finished
412/6431 finished
413/6431 finished
414/6431 finished
415/6431 finished
416/6431 finished
417/6431 finished
418/6431 finished
419/6431 finished
420/6431 finished
421/6431 finished
422/6431 finished
423/6431 finished
424/6431 finished
425/6431 finished
426/6431 finished
427/6431 finished
428/6431 finished
429/6431 finished
430/6431 finished
431/6431 finished
432/6431 finished
433/6431 finished
434/6431 finished
435/6431 finished
436/6431 finished
437/6431 finished
438/6431 finished
439/6431 finished
440/6431 finished
441/6431 finished
442/6431 finished
443/6431 finished
444/6431 finished
445/6431 finished
446/6431 finished
447/6431 finished
448/6431 finished
449/6431 finished
450/6431 finished
451/6431 finished
452/6431 finished
453/6431 finished
454/6431 finished
455/6431 finished
456/6431 finished
457/6431 finished
458/6431 finished
459/6431 finished
460/6431 finished
461/6431 finished
462/6431 finished
463/6431 f

864/6431 finished
865/6431 finished
866/6431 finished
867/6431 finished
868/6431 finished
869/6431 finished
870/6431 finished
871/6431 finished
872/6431 finished
873/6431 finished
874/6431 finished
875/6431 finished
876/6431 finished
877/6431 finished
878/6431 finished
879/6431 finished
880/6431 finished
881/6431 finished
882/6431 finished
883/6431 finished
884/6431 finished
885/6431 finished
886/6431 finished
887/6431 finished
888/6431 finished
889/6431 finished
890/6431 finished
891/6431 finished
892/6431 finished
893/6431 finished
894/6431 finished
895/6431 finished
896/6431 finished
897/6431 finished
898/6431 finished
899/6431 finished
900/6431 finished
901/6431 finished
902/6431 finished
903/6431 finished
904/6431 finished
905/6431 finished
906/6431 finished
907/6431 finished
908/6431 finished
909/6431 finished
910/6431 finished
911/6431 finished
912/6431 finished
913/6431 finished
914/6431 finished
915/6431 finished
916/6431 finished
917/6431 finished
918/6431 finished
919/6431 f

1304/6431 finished
1305/6431 finished
1306/6431 finished
1307/6431 finished
1308/6431 finished
1309/6431 finished
1310/6431 finished
1311/6431 finished
1312/6431 finished
1313/6431 finished
1314/6431 finished
1315/6431 finished
1316/6431 finished
1317/6431 finished
1318/6431 finished
1319/6431 finished
1320/6431 finished
1321/6431 finished
1322/6431 finished
1323/6431 finished
1324/6431 finished
1325/6431 finished
1326/6431 finished
1327/6431 finished
1328/6431 finished
1329/6431 finished
1330/6431 finished
1331/6431 finished
1332/6431 finished
1333/6431 finished
1334/6431 finished
1335/6431 finished
1336/6431 finished
1337/6431 finished
1338/6431 finished
1339/6431 finished
1340/6431 finished
1341/6431 finished
1342/6431 finished
1343/6431 finished
1344/6431 finished
1345/6431 finished
1346/6431 finished
1347/6431 finished
1348/6431 finished
1349/6431 finished
1350/6431 finished
1351/6431 finished
1352/6431 finished
1353/6431 finished
1354/6431 finished
1355/6431 finished
1356/6431 fi

...

4328/6431 finished
4329/6431 finished
4330/6431 finished
4331/6431 finished
4332/6431 finished
4333/6431 finished
4334/6431 finished
4335/6431 finished
4336/6431 finished
4337/6431 finished
4338/6431 finished
4339/6431 finished
4340/6431 finished
4341/6431 finished
4342/6431 finished
4343/6431 finished
4344/6431 finished
4345/6431 finished
4346/6431 finished
4347/6431 finished
4348/6431 finished
4349/6431 finished
4350/6431 finished
4351/6431 finished
4352/6431 finished
4353/6431 finished
4354/6431 finished
4355/6431 finished
4356/6431 finished
4357/6431 finished
4358/6431 finished
4359/6431 finished
4360/6431 finished
4361/6431 finished
4362/6431 finished
4363/6431 finished
4364/6431 finished
4365/6431 finished
4366/6431 finished
4367/6431 finished
4368/6431 finished
4369/6431 finished
4370/6431 finished
4371/6431 finished
4372/6431 finished
4373/6431 finished
4374/6431 finished
4375/6431 finished
4376/6431 finished
4377/6431 finished
4378/6431 finished
4379/6431 finished
4380/6431 fi

4760/6431 finished
4761/6431 finished
4762/6431 finished
4763/6431 finished
4764/6431 finished
4765/6431 finished
4766/6431 finished
4767/6431 finished
4768/6431 finished
4769/6431 finished
4770/6431 finished
4771/6431 finished
4772/6431 finished
4773/6431 finished
4774/6431 finished
4775/6431 finished
4776/6431 finished
4777/6431 finished
4778/6431 finished
4779/6431 finished
4780/6431 finished
4781/6431 finished
4782/6431 finished
4783/6431 finished
4784/6431 finished
4785/6431 finished
4786/6431 finished
4787/6431 finished
4788/6431 finished
4789/6431 finished
4790/6431 finished
4791/6431 finished
4792/6431 finished
4793/6431 finished
4794/6431 finished
4795/6431 finished
4796/6431 finished
4797/6431 finished
4798/6431 finished
4799/6431 finished
4800/6431 finished
4801/6431 finished
4802/6431 finished
4803/6431 finished
4804/6431 finished
4805/6431 finished
4806/6431 finished
4807/6431 finished
4808/6431 finished
4809/6431 finished
4810/6431 finished
4811/6431 finished
4812/6431 fi

5192/6431 finished
5193/6431 finished
5194/6431 finished
5195/6431 finished
5196/6431 finished
5197/6431 finished
5198/6431 finished
5199/6431 finished
5200/6431 finished
5201/6431 finished
5202/6431 finished
5203/6431 finished
5204/6431 finished
5205/6431 finished
5206/6431 finished
5207/6431 finished
5208/6431 finished
5209/6431 finished
5210/6431 finished
5211/6431 finished
5212/6431 finished
5213/6431 finished
5214/6431 finished
5215/6431 finished
5216/6431 finished
5217/6431 finished
5218/6431 finished
5219/6431 finished
5220/6431 finished
5221/6431 finished
5222/6431 finished
5223/6431 finished
5224/6431 finished
5225/6431 finished
5226/6431 finished
5227/6431 finished
5228/6431 finished
5229/6431 finished
5230/6431 finished
5231/6431 finished
5232/6431 finished
5233/6431 finished
5234/6431 finished
5235/6431 finished
5236/6431 finished
5237/6431 finished
5238/6431 finished
5239/6431 finished
5240/6431 finished
5241/6431 finished
5242/6431 finished
5243/6431 finished
5244/6431 fi

5624/6431 finished
5625/6431 finished
5626/6431 finished
5627/6431 finished
5628/6431 finished
5629/6431 finished
5630/6431 finished
5631/6431 finished
5632/6431 finished
5633/6431 finished
5634/6431 finished
5635/6431 finished
5636/6431 finished
5637/6431 finished
5638/6431 finished
5639/6431 finished
5640/6431 finished
5641/6431 finished
5642/6431 finished
5643/6431 finished
5644/6431 finished
5645/6431 finished
5646/6431 finished
5647/6431 finished
5648/6431 finished
5649/6431 finished
5650/6431 finished
5651/6431 finished
5652/6431 finished
5653/6431 finished
5654/6431 finished
5655/6431 finished
5656/6431 finished
5657/6431 finished
5658/6431 finished
5659/6431 finished
5660/6431 finished
5661/6431 finished
5662/6431 finished
5663/6431 finished
5664/6431 finished
5665/6431 finished
5666/6431 finished
5667/6431 finished
5668/6431 finished
5669/6431 finished
5670/6431 finished
5671/6431 finished
5672/6431 finished
5673/6431 finished
5674/6431 finished
5675/6431 finished
5676/6431 fi

6057/6431 finished
6058/6431 finished
6059/6431 finished
6060/6431 finished
6061/6431 finished
6062/6431 finished
6063/6431 finished
6064/6431 finished
6065/6431 finished
6066/6431 finished
6067/6431 finished
6068/6431 finished
6069/6431 finished
6070/6431 finished
6071/6431 finished
6072/6431 finished
6073/6431 finished
6074/6431 finished
6075/6431 finished
6076/6431 finished
6077/6431 finished
6078/6431 finished
6079/6431 finished
6080/6431 finished
6081/6431 finished
6082/6431 finished
6083/6431 finished
6084/6431 finished
6085/6431 finished
6086/6431 finished
6087/6431 finished
6088/6431 finished
6089/6431 finished
6090/6431 finished
6091/6431 finished
6092/6431 finished
6093/6431 finished
6094/6431 finished
6095/6431 finished
6096/6431 finished
6097/6431 finished
6098/6431 finished
6099/6431 finished
6100/6431 finished
6101/6431 finished
6102/6431 finished
6103/6431 finished
6104/6431 finished
6105/6431 finished
6106/6431 finished
6107/6431 finished
6108/6431 finished
6109/6431 fi

In [6]:
for folder in image_folders:
    imgs = os.listdir(data_path + folder)
    print(imgs[0].split('_')[1].split('.png')[0])

-0.0090
-0.0024
0.0001
-0.0004
-0.0185
-0.0079
0.0005
0.0022
0.0001
-0.0000
0.0320
0.0118
0.0056
-0.0008
-0.0000
-0.0000
0.0109
0.0006
-0.0010
0.0098
-0.1536
0.0001
-0.0033
0.0002
0.0000
0.0502
-0.0001
0.1110
-0.0213
-0.0005
-0.0091
-0.0048
-0.0012
-0.0000
-0.0019
0.0051
0.0517
0.0018
0.0001
-0.0006
-0.0266
-0.0324
0.0001
0.0345
0.0003
-0.0006
-0.0129
0.0001
-0.0215
0.0004
0.0005
-0.0228
-0.0076
-0.0165
0.0002
0.0160
0.0542
-0.0076
0.0003
0.0032
-0.0082
-0.0075
-0.0004
0.0002
0.0005
0.0040
0.0043
0.0614
-0.0002
-0.0001
-0.0211
-0.0299
0.0000
-0.0133
0.0501
0.0011
0.0320
0.0000
0.0320
0.0103
0.0213
-0.0230
-0.1742
-0.0003
0.0197
-0.0026
-0.0686
-0.0175
0.0116
0.0047
-0.0004
0.0172
-0.1781
0.0012
-0.0053
-0.0000
-0.0086
-0.0001
0.0014
0.0046
-0.0025
0.0041
-0.0189
-0.0119
-0.0066
0.0293
-0.0001
0.0008
0.0002
0.0001
-0.0067
0.0035
0.0097
0.0047
0.0003
0.0218
0.0080
-0.0014
0.0004
0.0011
0.0002
0.0914
-0.0003
0.0000
0.0117
0.0004
-0.0189
-0.0210
0.0011
-0.0204
-0.0000
0.0056
-0.0026
-0.015

0.0005
-0.0246
-0.0191
0.0005
0.0518
0.0035
0.0046
0.0000
-0.0061
0.0167
-0.0000
0.0000
-0.0004
0.0239
0.0197
0.0108
-0.0000
-0.0009
0.0012
0.0000
-0.0002
-0.0002
-0.0093
-0.0435
0.0297
0.0004
-0.0031
-0.0073
-0.0238
0.0063
-0.0000
0.0000
0.0051
-0.0004
-0.0045
0.0496
-0.0011
0.0127
0.0001
0.0001
-0.0000
0.0000
0.0009
-0.0012
0.0183
0.0304
-0.0002
0.0334
0.0727
-0.1208
0.0068
0.0281
0.0007
-0.0208
-0.0247
0.0012
-0.0092
0.0287
-0.0033
-0.0027
0.0002
-0.0010
0.0004
-0.0016
-0.0209
-0.0036
0.0048
-0.0010
0.0052
0.0096
0.0011
0.0048
0.0322
-0.0693
0.0047
-0.0022
0.0001
0.0000
0.0771
0.0041
-0.0040
-0.0020
0.0011
-0.0017
-0.0000
-0.0249
-0.0073
0.0022
-0.0001
-0.0001
-0.0073
-0.0004
0.0031
0.0205
-0.0087
-0.0003
-0.0004
0.0000
-0.0004
-0.0062
-0.0088
0.0001
-0.0000
0.0014
0.0688
-0.0001
0.0051
-0.0076
-0.0008
0.0134
0.0168
0.0429
-0.0257
0.0378
-0.0000
0.0001
0.0002
0.0000
0.0021
-0.0449
-0.0179
0.0409
0.0062
0.0012
0.0001
0.0181
0.0001
-0.0159
0.0114
-0.0189
-0.0218
0.0098
-0.1028
0.0051


0.0601
-0.0223
-0.0014
0.0000
0.0194
0.0076
0.0100
0.0011
-0.0135
0.0001
0.0096
-0.0399
-0.0014
-0.0209
-0.0010
-0.0121
0.0002
0.0031
0.0115
0.0046
-0.0023
-0.0088
0.0036
-0.0323
0.0011
0.0000
-0.0061
-0.0004
-0.0029
-0.1200
-0.0195
0.0035
-0.0188
-0.0011
0.0003
-0.0000
-0.0000
0.0012
-0.0000
0.0250
0.0290
-0.1097
0.0005
-0.0045
-0.0142
0.0024
0.0008
0.0010
0.0004
0.0011
0.0010
0.0003
-0.0000
0.0214
-0.1274
-0.0254
0.0542
-0.0006
0.0002
-0.0000
-0.0036
-0.0013
0.0001
-0.0228
0.0000
-0.0212
0.0257
0.0310
0.0014
0.0492
0.0012
-0.0001
0.0035
-0.0077
-0.0186
0.0061
-0.0240
0.0053
0.0668
-0.0023
0.0166
0.0042
0.0297
-0.0767
0.0400
-0.0001
-0.0033
0.0003
-0.0006
0.0063
0.0498
-0.0001
-0.0195
0.0011
0.0067
-0.0001
-0.0012
-0.0000
-0.0004
-0.0080
0.0116
-0.0023
-0.0231
-0.0000
-0.0001
0.0001
0.0650
0.0013
-0.0000
0.0008
-0.0036
0.0115
-0.0000
-0.0008
-0.0025
-0.0001
-0.0002
0.0048
0.0001
-0.0004
-0.0367
0.0035
0.0001
-0.0001
0.0079
-0.0005
0.0001
0.0080
-0.0094
0.0743
0.0082
-0.0441
0.0000
0.0

-0.0000
0.0739
-0.0023
0.0245
-0.0008
-0.0189
-0.0003
0.0008
-0.0194
-0.0089
-0.0185
0.0203
-0.0034
0.0253
0.0003
-0.0000
-0.0004
0.0164
0.0120
-0.0202
0.0021
0.0012
